In [31]:
import pandas as pd
import numpy as np
import os

In [32]:
train_files = [i for i in os.listdir('results/') if '_val_reg' in i]
test_files = [i for i in os.listdir('results/') if '_test_reg' in i]

train_files.remove('y_val_reg')
train_files = sorted(train_files)
test_files = sorted(test_files)

In [33]:
y_train = np.fromfile("results/y_val_reg", dtype='float32')

In [34]:
train = pd.DataFrame({"var" + str(i) : np.fromfile("results/" + file, dtype='float32') for i, file in enumerate(train_files)})
y = np.fromfile("results/y_val_reg", dtype='float32')

test = pd.DataFrame({"var" + str(i) : np.fromfile("results/" + file, dtype='float32') for i, file in enumerate(test_files)})


In [35]:
y_final_preds = np.round(np.clip(test, 1, 10).mean(axis=1)).astype(int)

In [36]:
TestX = pd.read_csv("data/TestX.csv", sep=";")
Train = pd.read_csv("data/Train.csv", sep=";")

Train = Train.drop_duplicates(['condition', 'opinion']).reset_index(drop=True)

submission = pd.read_csv("data/JANLOS.txt", sep=" ")

In [37]:
TestX_merged = TestX.merge(Train, on=['opinion', 'condition'], how='left')

In [38]:
TestX_merged['y_final_preds'] = y_final_preds

In [39]:
TestX_merged.head()

,name_x,condition,opinion,name_y,rate,rate1,y_final_preds
0,Vitamin B12,Vitamin B12 Deficiency,"""No after taste at unlike all the others that ...",Cyanocobalamin,10.0,high,8
1,Implanon,Birth Control,"""I have not had any issues with the implanon I...",Etonogestrel,8.0,high,8
2,BuSpar,Anxiety,"""I have been taking Buspar and Celexa and as f...",Buspirone,10.0,high,9
3,Abreva,Herpes Simplex,"""the only thing that works. completely awesome.""",Docosanol,10.0,high,10
4,Buprenorphine / naloxone,Opiate Dependence,"""UPDATE: I originally posted on Nov 16th 2010 ...",Suboxone,10.0,high,10


In [40]:
TestX_merged["rate_final"] = TestX_merged['rate'].fillna(TestX_merged['y_final_preds'])
TestX_merged.rate_final = TestX_merged.rate_final.astype(int)

In [46]:
TestX_merged.rate_final.value_counts()

10    13430
9     11405
8      5622
1      5134
7      2995
2      2788
3      2337
5      2292
6      2090
4      1907
Name: rate_final, dtype: int64

In [51]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
def voted_result(filepath, n_models, n_classes):
    
    y_hat_ensemble = None
    
    for i in range(n_models):
        file = filepath.replace('#', str(i))
        y_hat = np.fromfile(file, dtype='float32').reshape(-1, n_classes)
        print(np.argmax(y_hat, axis=1))
        y_hat = to_categorical(np.argmax(y_hat, axis=1), num_classes=n_classes)
            
        if y_hat_ensemble is None:
            y_hat_ensemble = np.zeros_like(y_hat)
        y_hat_ensemble = y_hat_ensemble + y_hat
    
    y_hat_ensemble = y_hat_ensemble + np.random.normal(size=y_hat_ensemble.shape) / 1e4
    
    return np.argmax(y_hat_ensemble, axis=1)

In [52]:
y_hat_val = voted_result('./results/Model_1_#_test_cls', 10, 3)

[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [53]:
y_hat_val

array([0, 0, 0, ..., 0, 0, 0])

In [54]:
d = {0: "low", 1: "medium", 2: "high"}

In [56]:
TestX_merged['rate1_preds'] = y_hat_val

In [57]:
TestX_merged['rate1_preds'] = TestX_merged['rate1_preds'].replace(d)

In [63]:
TestX_merged.rate1_preds.value_counts()

low       32916
medium    11226
high       5858
Name: rate1_preds, dtype: int64

In [61]:
np.bincount(y_hat_val)

array([32916, 11226,  5858])

In [62]:
TestX_merged["rate1_final"] = TestX_merged['rate1'].fillna(TestX_merged['rate1_preds'])

In [64]:
TestX_merged.rate1_final.value_counts()

low       21077
high      18923
medium    10000
Name: rate1_final, dtype: int64

In [67]:
output = {"rate": TestX_merged.rate_final, "rate1": TestX_merged.rate1_final}

In [69]:
output = pd.DataFrame(output)

In [71]:
output.to_csv("./JAKKAL.txt", sep=" ", index=False, qouting)